# DO NOT "RUN ALL"

### Create all LaTeX tables

In [15]:
import subprocess

commands = [
"python read_global_model_logs.py --filename logs/resnet_18_100_users.csv --dataset_filter cifar10 --partition iid",
# "python read_global_model_logs.py --filename logs/resnet_18_100_users.csv --dataset_filter cifar10 --partition noniid-labeldir",

"python read_global_model_logs.py --filename logs/resnet_18_100_users_200_rounds.csv --dataset_filter cifar10 --partition noniid-labeldir",

"python read_global_model_logs.py --filename logs/resnet_50_100_users_300_epochs.csv --dataset_filter cifar100 --partition iid",
"python read_global_model_logs.py --filename logs/resnet_50_100_users_300_epochs.csv --dataset_filter cifar100 --partition noniid-labeldir",

"python read_global_model_logs.py --filename logs/resnet_50_10_users_300_epochs.csv --dataset_filter cifar100 --partition iid",
"python read_global_model_logs.py --filename logs/resnet_50_10_users_300_epochs.csv --dataset_filter cifar100 --partition noniid-labeldir",

"python read_global_model_logs.py --filename logs/mobilenetv3_tinyimagenet.csv --dataset_filter tinyimagenet --partition iid",

"python read_global_model_logs.py --filename logs/resnet_50_10_users_tinyimagenet.csv --dataset_filter tinyimagenet --partition iid",
"python read_global_model_logs.py --filename logs/resnet_50_10_users_tinyimagenet.csv --dataset_filter tinyimagenet --partition noniid-labeldir",

"python read_global_model_logs.py --filename logs/resnet18_ham.csv --dataset_filter ham10000 --partition iid",
]

with open("output_tables.txt", "w") as file:
    for command in commands:
        output = subprocess.check_output(command, shell=True, universal_newlines=True)
        # print(output)
        file.write(output)
        file.write("\n")

### Manual deletion of specific runs

#### Step 1: Filter

In [26]:
# Cell 1: Filter runs based on hyperparameters
import pandas as pd
import ast

def filter_runs(filename, hyperparameter_filters):
    df = pd.read_csv(filename)
    
    def match_hyperparameters(row, filters):
        hyperparameters = ast.literal_eval(row['Hyperparameters'])
        for key, value in filters.items():
            if key not in hyperparameters or hyperparameters[key] != value:
                return False
        return True
    
    filtered_df = df[df.apply(lambda row: match_hyperparameters(row, hyperparameter_filters), axis=1)]
    
    for _, row in filtered_df.iterrows():
        hyperparameters = ast.literal_eval(row['Hyperparameters'])
        matched_hyperparameters = {key: hyperparameters[key] for key in hyperparameter_filters}
        print(f"Best Global Model Test: {row['Best Global Model Test']}")
        print(f"Matched Hyperparameters: {matched_hyperparameters}")
        print("---")
    
    return filtered_df

# Example usage
filename = 'logs/resnet_18_100_users_200_rounds.csv'
hyperparameter_filters = {
    "alg": "fedavg",
    "partition": "noniid-labeldir",
    "dataset": "cifar10",
    "optimizer": "sgd",
    "lr": 0.01,
    "batch_size": 128,
}

filtered_runs = filter_runs(filename, hyperparameter_filters)

Best Global Model Test: 0.6058
Matched Hyperparameters: {'alg': 'fedavg', 'partition': 'noniid-labeldir', 'dataset': 'cifar10', 'optimizer': 'sgd', 'lr': 0.01, 'batch_size': 128}
---
Best Global Model Test: 0.6063
Matched Hyperparameters: {'alg': 'fedavg', 'partition': 'noniid-labeldir', 'dataset': 'cifar10', 'optimizer': 'sgd', 'lr': 0.01, 'batch_size': 128}
---
Best Global Model Test: 0.5531
Matched Hyperparameters: {'alg': 'fedavg', 'partition': 'noniid-labeldir', 'dataset': 'cifar10', 'optimizer': 'sgd', 'lr': 0.01, 'batch_size': 128}
---


#### Step 2: Delete filtered rows

In [27]:
# Cell 2: Delete matching runs from the CSV file
import pandas as pd

def delete_matching_runs(filename, filtered_runs):
    df = pd.read_csv(filename)
    updated_df = df[~df.index.isin(filtered_runs.index)]
    updated_df.to_csv(filename, index=False)
    print(f"Deleted {len(filtered_runs)} matching runs from the CSV file.")

# Example usage
delete_matching_runs(filename, filtered_runs)

Deleted 3 matching runs from the CSV file.
